In [1]:
from google.cloud import bigquery
bq = bigquery.Client(project="project-2-308905")
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [2]:
#q1
q1 = bq.query("""
SELECT geo_id, county_name
FROM `bigquery-public-data.geo_us_boundaries.counties`
WHERE county_name = "Dane"
LIMIT 10
""", job_config=no_cache)
df1 = q1.to_dataframe()
geo_id = df1["geo_id"][0]
geo_id

'55025'

In [3]:
#q2
q2 = bq.query("""
SELECT state_fips_code, COUNT(county_name) as count
FROM `bigquery-public-data.geo_us_boundaries.counties` 
GROUP BY state_fips_code
ORDER BY count DESC
LIMIT 5
""", job_config=no_cache)
df2 = q2.to_dataframe()
dict(zip(df2["state_fips_code"], df2["count"]))

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [4]:
#q3
costs = {}
q1_cost = 5 * (q1.total_bytes_billed / 1024**4)
q2_cost = 5 * (q2.total_bytes_billed / 1024**4)
costs["q1"] = q1_cost
costs["q2"] = q2_cost
costs

{'q1': 4.76837158203125e-05, 'q2': 4.76837158203125e-05}

In [5]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
job = bq.load_table_from_uri(["gs://apr29/hdma-wi-2021.parquet"],
                             "project-2-308905.p7.hdma", job_config=config)

In [6]:
#q4
dataset = bigquery.Dataset("project-2-308905.p7")
bq.create_dataset(dataset, exists_ok=True)
[ds.dataset_id for ds in bq.list_datasets("project-2-308905")]

['p7']

In [7]:
#q5
q5 = bq.query("""
SELECT county_name, COUNT(*) as count
FROM bigquery-public-data.geo_us_boundaries.counties as tb1
INNER JOIN project-2-308905.p7.hdma as tb2
ON tb1.county_fips_code = tb2.county_code
GROUP BY county_name
ORDER BY count DESC
LIMIT 10
""", job_config=no_cache)
df5 = q5.to_dataframe()
dict(zip(df5["county_name"], df5["count"]))

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [8]:
url = "https://drive.google.com/open?id=1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(dataset.table("applications"))
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [9]:
#q6
q6 = bq.query("""
SELECT COUNT(*) as count
FROM project-2-308905.p7.applications
WHERE income = 100000
""", job_config=no_cache)
df6 = q6.to_dataframe()
df6["count"][0]

11

In [10]:
wi = bq.query("""
CREATE OR REPLACE TABLE project-2-308905.p7.wi
AS

SELECT county_name, county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55' -- is WI
""")

homes = bq.query("""
CREATE OR REPLACE TABLE project-2-308905.p7.homes
AS

SELECT *, ST_GEOGPOINT(longitude, latitude) AS loc
FROM project-2-308905.p7.applications
WHERE (latitude BETWEEN 40 and 60) AND (longitude BETWEEN -100 and -75)
""")

In [11]:
#q7
q7 = bq.query("""
SELECT county_name, COUNT(*) as count
FROM project-2-308905.p7.homes, project-2-308905.p7.wi
WHERE ST_WITHIN(homes.loc, wi.county_geom)
GROUP BY county_name
""", job_config=no_cache)
df7 = q7.to_dataframe()
dict(zip(df7["county_name"], df7["count"]))

{'Columbia': 1,
 'Sheboygan': 1,
 'Oneida': 1,
 'Adams': 1,
 'Bayfield': 2,
 'Green Lake': 1,
 'Jefferson': 2,
 'Monroe': 1,
 'La Crosse': 1,
 'Barron': 1,
 'Brown': 3,
 'Kewaunee': 1,
 'Sauk': 1,
 'Door': 4,
 'Dane': 22,
 'Walworth': 2,
 'Waukesha': 1,
 'Milwaukee': 1}

In [12]:
model = bq.query("""
CREATE OR REPLACE MODEL p7.mymodel
OPTIONS(model_type='LINEAR_REG', INPUT_LABEL_COLS=['loan_amount'])

AS

SELECT income, loan_term, loan_amount
FROM p7.hdma
""")
model.result()

In [13]:
#q8
m = bq.query("""
SELECT *
FROM
ML.EVALUATE(
    MODEL p7.mymodel,
    (
        SELECT income, loan_term, loan_amount
        FROM p7.hdma
    )
)
""")
model_df = m.to_dataframe()
model_df.iloc[0].at["r2_score"]

0.29165412288020376

In [14]:
#q9
weights_df = bq.query("""
SELECT * FROM
ML.WEIGHTS(MODEL `p7.mymodel`)
""").to_dataframe()
weights_df.iloc[0]["weight"]

0.8057773037176671

In [15]:
#q10
predictions = bq.query("""
SELECT 
  CASE WHEN loan_amount > predicted_loan_amount 
    THEN 1 
    ELSE 0 
  END
  AS new_col
FROM
ML.PREDICT(
    MODEL p7.mymodel,
    (
        SELECT income, loan_amount, 360 AS loan_term
        FROM p7.applications
    )
)
""")
predictions_df = predictions.to_dataframe()
predictions_df['new_col'].mean()

0.6086956521739131